Anthropic supports tools, allowing you to easily connect with different external APIs and the like!

For this example, we are going to make a tool called "Cyberware" which will be our tool; we will create a dummy API that gives specs for cyberware from a specific company before the LLM says if the cyberware is good for a build. To do so, we will use both the tool system and an async function!

First, we start by importing Agentops and Anthropic

In [3]:
!pip install agentops
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.3/287.3 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: psutil
    Found existing installation: psutil 5.9.3
    Uninstalling psutil-5.9.3:
      Successfully uninstalled psutil-5.9.3
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires

Setup our generic default statements

In [4]:
from anthropic import Anthropic, AsyncAnthropic
import agentops
import os
import random  # We don't need this for agentops, we use this to generate a message later
import time  # We don't need this for agentops either, we use this when simulating an API later
import re  # Regex for formatting
from dotenv import load_dotenv

And set our API keys.

In [5]:
load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY") or "ANTHROPIC API KEY"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "AGENTOPS API KEY"


Now let's set the client as Anthropic and start an AgentOps session

In [ ]:
agentops.init(AGENTOPS_API_KEY, default_tags=["anthropic-example-tool"])

In [6]:
client = Anthropic(api_key=ANTHROPIC_API_KEY)

Let's create a dummy tool now! First off, let's create a list of companies for the system to choose from

In [9]:
Corpo = [
    "Kiroshi",
    "Arasaka",
    "Kang Tao",
    "Militech",
    "Biotechnica",
    "Zetatech",
    "Dynalar",
]

And now we create a DB! This could be anything from an .xml/.json to Postgres or MySQL! For our intent of a test, we will just include a dictionary.

In [55]:
cyberware_list = [
    {
        "name": "Kiroshi Optics",
        "creator": "Kiroshi",
        "bio": "Advanced cybernetic eye implants providing enhanced vision, a heads-up display (HUD), and scanning capabilities.",
        "stats": {
            "vision_modes": ["Thermal", "Night Vision", "Zoom"],
            "target_analysis": "Enemy Weak Spots Highlighted",
        },
    },
    {
        "name": "Gorilla Arms",
        "creator": "Arasaka",
        "bio": "Mechanical arms designed to enhance physical strength, allowing for powerful melee attacks and the ability to rip open doors.",
        "stats": {"melee_damage_bonus": "+100%", "strength_check_bonus": "+20"},
    },
    {
        "name": "Mantis Blades",
        "creator": "Arasaka",
        "bio": "Arm-mounted blades used for close combat, retractable from the forearms and capable of delivering high-speed slashes.",
        "stats": {
            "damage_type": "Physical",
            "attack_speed": "+30%",
            "bleed_chance": "50%",
        },
    },
    {
        "name": "Monowire",
        "creator": "Kang Tao",
        "bio": "A high-tech fiber-optic whip weapon that can slice through enemies with ease and can be charged for extra damage.",
        "stats": {
            "damage_type": "Physical/Electrical",
            "charge_bonus_damage": "+200%",
            "range": "5 meters",
        },
    },
    {
        "name": "Projectile Launch System",
        "creator": "Militech",
        "bio": "An arm-mounted cannon that allows the user to launch various types of projectiles, including grenades and explosive rounds.",
        "stats": {
            "ammo_types": ["Explosive", "Incendiary", "EMP"],
            "blast_radius": "3 meters",
        },
    },
    {
        "name": "Syn-Lungs",
        "creator": "Biotechnica",
        "bio": "Synthetic lungs that improve the user's breathing efficiency and stamina, allowing for prolonged physical exertion.",
        "stats": {"stamina_regen_rate": "+25%", "oxygen_capacity": "+30%"},
    },
    {
        "name": "Reinforced Tendons",
        "creator": "Arasaka",
        "bio": "Muscular enhancements that allow the user to jump higher and perform acrobatic movements.",
        "stats": {"jump_height": "+2 meters", "stamina_cost_reduction": "20%"},
    },
    {
        "name": "Bionic Joints",
        "creator": "Zetatech",
        "bio": "Cybernetic enhancements to the joints, providing increased flexibility and limb strength.",
        "stats": {"mobility_increase": "+15%", "limb_strength": "+20"},
    },
    {
        "name": "Subdermal Armor",
        "creator": "Militech",
        "bio": "Under-the-skin armor implants that increase the user's resistance to damage.",
        "stats": {"armor_bonus": "+200", "damage_resistance": "20%"},
    },
    {
        "name": "Sandevistan",
        "creator": "Dynalar",
        "bio": "Reflex booster that slows down time for the user, allowing them to move at superhuman speed.",
        "stats": {
            "duration": "8 seconds",
            "cooldown": "30 seconds",
            "speed_increase": "+50%",
        },
    },
    {
        "name": "Berserk",
        "creator": "Arasaka",
        "bio": "An adrenaline-inducing cyberware that temporarily boosts the user's physical capabilities, including strength and damage resistance.",
        "stats": {
            "duration": "10 seconds",
            "strength_bonus": "+30",
            "damage_reduction": "15%",
        },
    },
    {
        "name": "Cyberdeck",
        "creator": "NetWatch",
        "bio": "Cybernetic interface used for hacking, allowing the user to deploy quickhacks and breach enemy systems.",
        "stats": {"RAM_slots": "8", "quickhack_upload_speed": "+30%"},
    },
    {
        "name": "Pain Editor",
        "creator": "Biotechnica",
        "bio": "A neurological implant that reduces the perception of pain, allowing the user to endure more damage.",
        "stats": {"damage_taken_reduction": "10%", "bleed_resistance": "+50%"},
    },
    {
        "name": "Blood Pump",
        "creator": "Arasaka",
        "bio": "An enhanced circulatory system that improves health regeneration during combat.",
        "stats": {"health_regen_per_second": "+5%", "activation_duration": "6 seconds"},
    },
    {
        "name": "Heal-On-Kill",
        "creator": "Militech",
        "bio": "A system that restores a portion of the user's health each time they defeat an enemy.",
        "stats": {"health_restored_per_kill": "20%", "cooldown": "3 seconds"},
    },
    {
        "name": "Smart Link",
        "creator": "Kang Tao",
        "bio": "A wrist implant that increases smart weapon accuracy and locks onto targets for better aim.",
        "stats": {"smart_weapon_accuracy": "+15%", "target_lock_speed": "+25%"},
    },
    {
        "name": "Nano Relays",
        "creator": "Zetatech",
        "bio": "Enhances the duration of Sandevistan or Berserk by boosting neural signal transmission.",
        "stats": {"duration_increase": "+2 seconds"},
    },
    {
        "name": "Optical Camo",
        "creator": "Militech",
        "bio": "A cloaking device that provides temporary invisibility to the user.",
        "stats": {"invisibility_duration": "10 seconds", "cooldown": "30 seconds"},
    },
    {
        "name": "Bioconductor",
        "creator": "Biotechnica",
        "bio": "Regulates the body’s internal processes, reducing cyberware cooldowns.",
        "stats": {"cooldown_reduction": "20%"},
    },
    {
        "name": "Second Heart",
        "creator": "Biotechnica",
        "bio": "A failsafe biological implant that revives the user once when they die.",
        "stats": {"revive_health": "50%", "cooldown": "2 minutes"},
    },
    {
        "name": "Biomonitor",
        "creator": "Arasaka",
        "bio": "Monitors the user's vital signs and triggers healing if health drops too low.",
        "stats": {"activation_threshold": "15% health", "healing_amount": "30%"},
    },
    {
        "name": "Neofiber",
        "creator": "Zetatech",
        "bio": "A muscle fiber enhancement that increases evasion and movement speed.",
        "stats": {"evasion_increase": "+10%", "movement_speed_bonus": "+5%"},
    },
    {
        "name": "Cataresist",
        "creator": "Biotechnica",
        "bio": "Improves the user's resistance to toxins and shock-based damage.",
        "stats": {"poison_resistance": "+30%", "shock_resistance": "+25%"},
    },
    {
        "name": "Microgenerator",
        "creator": "Militech",
        "bio": "Releases a shockwave when the user takes damage, knocking back enemies.",
        "stats": {"shockwave_radius": "3 meters", "cooldown": "10 seconds"},
    },
    {
        "name": "Fortified Ankles",
        "creator": "Dynalar",
        "bio": "Reinforces the legs to allow for charged jumps and enhanced mobility.",
        "stats": {"charged_jump_height": "+3 meters", "stamina_cost": "15%"},
    },
    {
        "name": "Reflex Tuner",
        "creator": "Arasaka",
        "bio": "Slows time when the user's health falls below a certain level, providing a brief window for recovery.",
        "stats": {"activation_threshold": "25% health", "duration": "5 seconds"},
    },
    {
        "name": "Techgogs",
        "creator": "Kiroshi",
        "bio": "Enhanced goggles that provide better target acquisition and analysis.",
        "stats": {"targeting_accuracy": "+20%", "analyze_speed": "+25%"},
    },
]

And finally we make build types! We will keep to 6 based off the Edgerunners anime! (Try guessing which archetype is who)

In [11]:
edgerunners_builds = [
    {
        "name": "The Agile Fighter",
        "description": "An adaptable build that focuses on speed and reflexes. Excelling in close-quarters combat, this fighter utilizes a mix of melee and ranged weapons, quickly adapting to any situation and embodying the spirit of a street fighter.",
    },
    {
        "name": "The Chaotic Gunner",
        "description": "An unpredictable build specializing in dual-wielding firearms and fast-paced combat. With high energy and a penchant for mayhem, this gunner overwhelms enemies with speed and accuracy, making them a force to be reckoned with in any firefight.",
    },
    {
        "name": "The Heavy Brawler",
        "description": "A build centered around strength and durability, combining powerful weaponry with advanced cybernetic enhancements. This brawler can absorb damage and unleash devastating attacks, making them a formidable frontline combatant.",
    },
    {
        "name": "The Stealth Hacker",
        "description": "A stealthy and agile build focused on evasion and hacking. Utilizing advanced cyberware, this infiltrator manipulates the environment and sneaks past enemy lines without detection, excelling in covert operations.",
    },
    {
        "name": "The Tactical Defender",
        "description": "A defensive build that excels in protecting allies and providing support on the battlefield. With resilience and tactical prowess, this defender can absorb enemy fire while counterattacking, ideal for players who prefer a supportive role.",
    },
    {
        "name": "The Supportive Strategist",
        "description": "A build focused on enhancing team performance. Utilizing a mix of hacking and buffing skills, this strategist provides advantages in combat, ensuring that allies remain strong and focused during engagements.",
    },
]

Now that that's done, we can make a function! We will take the input from the user; for this test, we can assume we will always get a proper corpo name. We have a two second wait to simulate an online API request

In [13]:
def get_cyberware_by_creator(creator_name):
    # Filter the items by creator name (case-insensitive)
    filtered_items = [
        item
        for item in cyberware_list
        if item["creator"].lower() == creator_name.lower()
    ]

    # If there are no items found, handle it appropriately
    if not filtered_items:
        return "No cyberware found for this creator."

    # Select a random item from the filtered list
    returned_item = random.choice(filtered_items)

    # Pause for 2 seconds (simulate some kind of delay)
    time.sleep(2)

    # Create a final formatted string to return
    final = f'Name: {returned_item["name"]}, Creator: {returned_item["creator"]}, Bio: {returned_item["bio"]}, Stats: {returned_item["stats"]}'

    return final

In [14]:
get_cyberware_by_creator("Militech")

"Name: Projectile Launch System, Creator: Militech, Bio: An arm-mounted cannon that allows the user to launch various types of projectiles, including grenades and explosive rounds., Stats: {'ammo_types': ['Explosive', 'Incendiary', 'EMP'], 'blast_radius': '3 meters'}"

Now to the real core of this; making our message stream! We create this as a function we can call later! I create examples since the LLM's context size can handle it!

We are also going to take several steps here; we must create an example of the tool being used as context. Next, we must add the generated lines to the messages list once done being generated. Finally, we will parse the text for the format we want and request another line

Now we make a message!

In [15]:
# We make our history a separate block to be easier to add to and get a random build to begin

# Get a random build
random_build = random.choice(edgerunners_builds)

# We make our history a separate block to be easier to add to and get a random build to begin
initialmessages = [
    {
        "role": "user",
        "content": "The Heavy Tank - This build focuses on durability and defense, sacrificing speed for maximum resilience. Ideal for handling sustained combat and enduring powerful hits. Requested corporation is Arasaka.",
    },
    {
        "role": "assistant",
        "content": "Starting Search! get_cyberware_by_creator[Arasaka]",
    },
    {
        "role": "assistant",
        "content": "Name: Reinforced Subdermal Armor, Creator: Arasaka, Bio: Provides additional layers of armor under the skin to absorb heavy impact, Stats: +50% physical damage resistance, -10% agility",
    },
    {
        "role": "assistant",
        "content": "The Reinforced Subdermal Armor is a solid choice for The Heavy Tank build. The additional 50% damage resistance will allow you to endure powerful attacks, which is perfect for your defensive style. Be aware of the slight agility decrease, though it aligns with your build's focus on resilience over speed.",
    },
    {
        "role": "user",
        "content": "The Silent Assassin - A stealth-focused build that emphasizes silent takedowns and avoiding detection. This assassin utilizes a combination of stealth and high precision in ranged combat. Requested corporation is Militech.",
    },
    {
        "role": "assistant",
        "content": "Starting Search! get_cyberware_by_creator[Militech]",
    },
    {
        "role": "assistant",
        "content": "Name: Optical Camouflage System, Creator: Militech, Bio: A cloaking system that renders the user nearly invisible for a short duration, Stats: Duration: 10 seconds, Cooldown: 40 seconds",
    },
    {
        "role": "assistant",
        "content": "The Optical Camouflage System is highly suitable for The Silent Assassin build. Its invisibility feature will allow you to bypass enemies and perform stealth takedowns without being noticed. Just be cautious of its cooldown and plan your escape if needed.",
    },
    {
        "role": "user",
        "content": "The Tech Savant - A build that excels in using advanced technology and gadgets, specializing in hacking and controlling devices. This tech expert uses drones and automated turrets in combat. Requested corporation is Zetatech.",
    },
    {
        "role": "assistant",
        "content": "Starting Search! get_cyberware_by_creator[Zetatech]",
    },
    {
        "role": "assistant",
        "content": "Name: Drone Control Interface, Creator: Zetatech, Bio: Allows the user to control multiple drones simultaneously and provides an enhanced HUD for drone operations, Stats: Drone Capacity: +2, HUD enhancement for better drone tracking",
    },
    {
        "role": "assistant",
        "content": "The Drone Control Interface is ideal for The Tech Savant build, as it increases your ability to control multiple drones and improves your situational awareness through the enhanced HUD. This item will amplify your tech-based combat strategy significantly.",
    },
    {
        "role": "user",
        "content": "The Cyber Sniper - A long-range build specializing in precision and high-damage shots from a distance. This sniper focuses on accuracy and stability. Requested corporation is Kang Tao.",
    },
    {
        "role": "assistant",
        "content": "Starting Search! get_cyberware_by_creator[Kang Tao]",
    },
    {
        "role": "assistant",
        "content": "Name: Stabilizer Arms, Creator: Kang Tao, Bio: Cybernetic arms with built-in stabilizers to reduce weapon sway, Stats: Recoil Reduction: 80%, Increased Precision: +30%",
    },
    {
        "role": "assistant",
        "content": "The Stabilizer Arms are an excellent choice for The Cyber Sniper. The enhanced precision and recoil reduction will improve your accuracy, making it easier to land precise, high-damage shots from long range. Perfect for maintaining a steady aim during extended engagements.",
    },
    {
        "role": "user",
        "content": f"Based on the user's build type and requested corporation, get a random item from the corporation and tell if it will be a good idea to use; {random_build['name']} - {random_build['description']}, Requested Coroporation is {random.choice(Corpo)}",
    },
]

For now, this is where we will talk to the LLM! Stream on seems to have a few problems so we get the output as one chunk! 

In [16]:
response = client.messages.create(
    max_tokens=5000,
    model="claude-3-5-sonnet-20240620",
    tools=[
        {
            "name": "get_cyberware_by_creator",
            "description": "Retrieve cyberware information based on the manufacturer corporation",
            "input_schema": {
                "type": "object",
                "properties": {
                    "creator": {
                        "type": "string",
                        "description": "The name of the cyberware creator",
                    }
                },
                "required": ["creator"],
            },
        }
    ],
    messages=initialmessages,
)

print(response)

Message(id='msg_01Afykywq3GD2EoHaGGxujRy', content=[TextBlock(text="Certainly! Let's see what Zetatech has to offer for The Stealth Hacker build. I'll retrieve a random item from Zetatech and evaluate its suitability for your build.", type='text'), ToolUseBlock(id='toolu_011JwtgPzy3a88NR1s9BmgNL', input={'creator': 'Zetatech'}, name='get_cyberware_by_creator', type='tool_use')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=1206, output_tokens=109))


Now to get the tool used! We know it will be used and can pretty much ignore the AI's text, still let's show it anyways! We are going to see if the tool name is called get_cyberware_by_creator (And we know it's going to %99 be there)

In [47]:
SearchResult = ""

# Print response content to see the data
print(response.content)

# Assuming ToolUseBlock is at index 1
tool_use_block = response.content[1]

# Get the tool name and input
tool_name = tool_use_block.name
tool_input = tool_use_block.input

# Extract creator (e.g., 'Militech')
tool_creator = tool_input["creator"]

# Check if the tool name is "get_cyberware_by_creator"
if tool_name == "get_cyberware_by_creator":
    # Call the function with the tool creator as an argument
    SearchResult = get_cyberware_by_creator(tool_creator)

[TextBlock(text="Certainly! Let's see what Zetatech has to offer for The Stealth Hacker build. I'll retrieve a random item from Zetatech and evaluate its suitability for your build.", type='text'), ToolUseBlock(id='toolu_011JwtgPzy3a88NR1s9BmgNL', input={'creator': 'Zetatech'}, name='get_cyberware_by_creator', type='tool_use')]


Now, we will add an item to the history and create our next request!

In [49]:
initialmessages.append({"role": "assistant", "content": SearchResult})

initialmessages.append({"role": "user", "content": "Is this a good match?"})

Now let's see if this is good!

In [50]:
response = client.messages.create(
    max_tokens=5000,
    model="claude-3-5-sonnet-20240620",
    tools=[
        {
            "name": "get_cyberware_by_creator",
            "description": "Retrieve cyberware information based on the manufacturer corporation",
            "input_schema": {
                "type": "object",
                "properties": {
                    "creator": {
                        "type": "string",
                        "description": "The name of the cyberware creator",
                    }
                },
                "required": ["creator"],
            },
        }
    ],
    messages=initialmessages,
)

print(response)

Message(id='msg_01YWrpDbF475reehYXHNv7nE', content=[TextBlock(text='Yes, the Neofiber cyberware from Zetatech is an excellent match for The Stealth Hacker build you described. Let\'s break down why:\n\n1. Alignment with build focus: The Stealth Hacker is described as a "stealthy and agile build focused on evasion." The Neofiber directly enhances both stealth and agility aspects of this build.\n\n2. Evasion increase: The +10% evasion increase is particularly valuable for a character that aims to avoid detection and enemy attacks. This will make it easier to sneak past enemy lines and escape if spotted.\n\n3. Movement speed bonus: The +5% movement speed bonus complements the build\'s agility focus. Faster movement allows for quicker repositioning, easier escapes, and more efficient navigation through complex environments during covert operations.\n\n4. Synergy with hacking: While this cyberware doesn\'t directly enhance hacking abilities, the improved evasion and speed will help the char

Now to display the output!

In [54]:
message = response.content[0].text
print(message)

Yes, the Neofiber cyberware from Zetatech is an excellent match for The Stealth Hacker build you described. Let's break down why:

1. Alignment with build focus: The Stealth Hacker is described as a "stealthy and agile build focused on evasion." The Neofiber directly enhances both stealth and agility aspects of this build.

2. Evasion increase: The +10% evasion increase is particularly valuable for a character that aims to avoid detection and enemy attacks. This will make it easier to sneak past enemy lines and escape if spotted.

3. Movement speed bonus: The +5% movement speed bonus complements the build's agility focus. Faster movement allows for quicker repositioning, easier escapes, and more efficient navigation through complex environments during covert operations.

4. Synergy with hacking: While this cyberware doesn't directly enhance hacking abilities, the improved evasion and speed will help the character get into optimal positions for hacking operations and quickly retreat if 

Run this to end the session!

In [ ]:
agentops.end_session("Success")